In [ ]:
class BruteForce:

    """
    This is a simple brute-force substring search for DNA sequences.
    Checks every starting position in the text.
    Returns all match positions
    """
    def search(self, pattern, txt):
        M = len(pattern)
        N = len(txt)
        matches = []

        #Check every starting position
        for i in range(N - M + 1):
            j = 0

            # Compare the characters one by one
            while j < M and txt[i + j] == pattern[j]:
                j += 1

            #If we matched the full pattern, record the index
            if j == M:
                matches.append(i)
        return matches
    #Returns true if pattern is appears at least 1 time
    def contains(self, pattern, text):
        return len(self.search(pattern, text)) > 0
           
